In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import tushare as ts

In [2]:
#交易期
trade_start_date = '20140101' 
trade_end_date = '20211227'
#股票代码
#ts_code = '600050.SH'
pro = ts.pro_api()
#hgt_ts_code = pro.hs_const(hs_type='SH')
sgt_ts_code = pro.hs_const(hs_type='SZ')

th_h = 20
th_m = 10
th_l = 5

In [23]:
def smaCal(tsPrice,k): #简单平均
    Sma = pd.Series(0.0,index=tsPrice.index)
    for i in range(k-1,len(tsPrice)):
        Sma[i]=sum(tsPrice[(i-k+1):(i+1)])/k
    return Sma

In [3]:
sgt_ts_code

,ts_code,hs_type,in_date,out_date,is_new
0,002910.SZ,SZ,20171114,None,1
1,000016.SZ,SZ,20180102,None,1
2,001872.SZ,SZ,20180102,None,1
3,000040.SZ,SZ,20180102,None,1
4,000401.SZ,SZ,20180102,None,1
...,...,...,...,...,...
237,300723.SZ,SZ,20191216,None,1
238,300751.SZ,SZ,20191216,None,1
239,300755.SZ,SZ,20191216,None,1
240,300759.SZ,SZ,20191223,None,1


In [24]:
def smaCal_low(tsPrice,tsPrice_low,k): #过去的收盘价和当天最低价计算简单平均
    Sma_low = pd.Series(0.0,index=tsPrice.index)
    for i in range(k-1,len(tsPrice)):
        Sma_low[i]=sum(tsPrice[(i-k+1):i]+tsPrice_low[i])/k
    return Sma_low

In [25]:
def smaCal_low(tsPrice,tsPrice_high,k): #过去的收盘价和当天最高价计算简单平均
    Sma_high = pd.Series(0.0,index=tsPrice.index)
    for i in range(k-1,len(tsPrice)):
        Sma_high[i]=sum(tsPrice[(i-k+1):i]+tsPrice_high[i])/k
    return Sma_high

In [26]:
def ewmaCal(tsPrice,period=5,exp=0.2):#加权指数平均
    Ewma = pd.Series(0.0,index=tsPrice.index)
    Ewma[period-1]=np.mean(tsPrice[:period])
    for i in range(period,len(tsPrice)):
        Ewma=exp*tsPrice[i]+(1-exp)*Ewma[i-1]
    return Ewma

In [27]:
def macdCal(tsPrice):#MACD
    DIF = ewmaCal(tsPrice,12,2/(1+12))-ewmaCal(tsPrice,26,2/(1+26))
    DEA = ewmaCal(DIF,9,2/1+9)
    MACD = DIF-DEA
    return DIF,DEA,MACD

In [28]:
def BreakPoint(tsPrice,kS,kL):
    bp = pd.Series(0.0,index=tsPrice.index)
    for i in range(kL-1,len(tsPrice)): 
        bp[i]=sum(tsPrice[i-kL+1:i-kS+1])-sum(tsPrice[i-kS+1:i])
    return bp                 

In [29]:
def smaSignal(SmaS,SmaL,price_low,price_high,BreakPoint):
    Signal = pd.Series(0.0,index=SmaS.index)
    for i in range(1,len(SmaS)):       
        #if SmaS[i-1] < SmaL[i-1] and BreakPoint[i] >= price_low[i] and BreakPoint[i] <= price_high[i] :#盘中突破
        if SmaS[i-1] < SmaL[i-1] and SmaS[i] > SmaL[i] :#收盘突破
            Signal[i] = 1
        elif SmaS[i-1] < SmaL[i-1] and BreakPoint[i] < price_low[i] :#高开过突破点
            Signal[i] = 2
        elif SmaS[i-1] > SmaL[i-1] and BreakPoint[i] >= price_low[i] and BreakPoint[i] <= price_high[i] :#盘中突破
            Signal[i] = -1
        elif SmaS[i-1] > SmaL[i-1] and BreakPoint[i] > price_high[i] :#低开过突破点
            Signal[i] = -2    
    return Signal     

In [30]:
def trade_sim(signal,bp,cash_init,price_low):
    cash = cash_init ;
    position = 0 
    share = 0 
    value = 0
    num = 0 
    for i in range(1,len(signal)):
        if position == 0 and signal[i] == 1:     #收盘买
            share = int(cash/bp[i]/100)
            cash = cash - share*bp[i]*100
            value = cash + bp[i]*share*100
            #print(bp[i],share*100,cash,value,num)
            position = 1
        elif position == 1 and signal[i] == -1:#收盘卖
            cash = cash + share*bp[i]*100            
            share = 0
            value = cash + share*bp[i]*100
            #print(bp[i],share*100,cash,value,num)
            position = 0
            num = num + 1
        elif position == 1 and signal[i] == -2:#开盘卖
            cash = cash + share*price_low[i]*100            
            share = 0
            value = cash + share*price_low[i]*100
            #print(price_high[i],share*100,cash,value,num)
            position = 0
            num = num + 1
    return value

In [ ]:
ts_sum = 0
win_rate = 0 
for ts_code in sgt_ts_code.ts_code:
    Unicom_data_trade = ts.pro_bar(ts_code=ts_code, adj='qfq', start_date=trade_start_date, end_date=trade_end_date)
    #倒序
    Unicom_data_trade=Unicom_data_trade.iloc[::-1]
    Unicom_data_trade.index=range(len(Unicom_data_trade))
    Sma5=smaCal(Unicom_data_trade.close,5)
    Sma10=smaCal(Unicom_data_trade.close,10)
    bp=BreakPoint(Unicom_data_trade.close,5,10)
    signal = smaSignal(Sma5,Sma10,Unicom_data_trade.low,Unicom_data_trade.high,bp)
    value = trade_sim(signal,bp,1000000,Unicom_data_trade.low)
    print(ts_code,(value-1000000)/1000000)
    win_rate = win_rate + (value-1000000)/1000000
    ts_sum = ts_sum + 1

002910.SZ 12.005518990000043
000016.SZ 168.83819348000003
001872.SZ 12.05078263000004
000040.SZ 96.92224184000005
000401.SZ 60.424914060000106
000429.SZ 28.41707554000012
000553.SZ 20.814500130000063
000560.SZ 63.660466189999944
000710.SZ 199.9436700000001
000723.SZ 243.61913527999994
000951.SZ 295.9017910000001
000968.SZ 78.90175509000001
002061.SZ 34.83787359999996
002507.SZ 141.00434720999976
002530.SZ 121.7534949000004
002537.SZ 58.488126609999945
002545.SZ 59.62802688000009
002611.SZ 359.9559895899999
002648.SZ 293.7132287499985
002747.SZ 192.29656540000005
002812.SZ 92.47322640999991
002839.SZ 3.414504460000016
002841.SZ 5.7551144300000265


In [19]:
win_rate

207.9678337700002

In [20]:
ts_sum

242

In [15]:
win_rate/ts_sum

0.7046162372289155